Based on work by https://github.com/opengeos/voila-geospatial/

My app running in https://huggingface.co/spaces/rosatrancoso/voila-geospatial/blob/main/notebooks/landcover_app.ipynb

To do: Resolve permissions

In [ ]:
import geemap
import ee

# if on google colab
# ee.Authenticate()
# ee.Initialize(project='test1-420603')

import pandas as pd
from io import StringIO

def rgb2hex(r,g,b):
    return "#{:02x}{:02x}{:02x}".format(r,g,b)

def hex2rgb(hexcode):
    return tuple(map(ord,hexcode[1:].decode('hex')))


# legends = geemap.builtin_legends
# for legend in legends:
#     print(legend)


In [ ]:
Map = geemap.Map(center=[-40,175], zoom=6)#, toolbar_ctrl=True, layer_ctrl=True)
Map

In [ ]:

# Add worldcover 10m
# worldcover2020_name = 'ESA WorldCover 2020 (10m)'
# worldcover2020 = ee.ImageCollection("ESA/WorldCover/v100").first()
# Map.addLayer(worldcover2020, {}, worldcover2020_name)


worldcover2021 = ee.ImageCollection("ESA/WorldCover/v200").first()
# Map.addLayer(worldcover2021, {}, 'ESA WorldCover 2021 (10m)')

# Map.add_legend(title="ESA WorldCover (10m)", builtin_legend="ESA_WorldCover")


In [ ]:
globcover2009 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select('landcover')

legend = pd.read_csv(StringIO("""
Red,Green,Blue,Class,Label
170,240,240,11,11 Post-flood/irrigated croplands (3)
255,255,100,14,14 Rainfed croplands (5)
220,240,100,20,20 Mosaic cropland 50-70%/vegetation 20-50% (9)
205,205,102,30,30 Mosaic cropland 50-70%/vegetation 20-50% (9)
0,100,0,40,40 >15% Evergreen broadleaf/semi-deciduous (11/13)
0,160,0,50,50 >40% Deciduous Broadleaf Forest (11)
170,200,0,60,60 15-40% Deciduous Broadleaf Forest(11)
0,60,0,70,70 >40% Evergreen Needleleaf Forest (14)
40,100,0,90,90 15-40% Needleleaf deciduous/evergreen (12/14)
120,130,0,100,100 >15% Broadleaf/NeedleLeaf (13/14)
140,160,0,110,110 50-70% forest/shrubland 20-50% grassland (13)
190,150,0,120,120 50-70% grassland 20-50% forest/shrubland (6)
150,100,0,130,130 >15% Shrubland (8)
255,180,50,140,140 >15% herbaceous vegetation (10)
255,235,175,150,150 Sparsely (<15%) Vegetated (19)
0,120,90,160,160 Regularly flooded broadleaf (18)
0,150,120,170,170 Permanently flooded broadleaf/shrubland (17)
0,220,130,180,180 Regularly flooded grassland/woody vegetation (3/17)
195,20,0,190,190 Artificial (1)
255,245,215,200,200 Barren (19)
0,70,200,210,210 Water Bodies (16)
255,255,255,220,220 Snow or Ice (24)
"""))

legend['Color'] = legend[['Red', 'Green', 'Blue']].apply(lambda x: rgb2hex(*x), axis=1)

vis_param = {
    "min": 11,
    "max": 220,
    "palette": list(legend['Color']),
}

legend_dict = {}
for index, row in legend.iterrows():
  legend_dict[row['Label']] = row['Color']


# Map.addLayer(globcover2009, vis_param, globcover2009_name)
# Map.add_legend(title='ESA Globcover 2009 (300 m)', legend_dict=legend_dict, position='bottomleft')



In [ ]:
left_layer = geemap.ee_tile_layer(globcover2009, vis_param, 'ESA Globcover 2009 (300 m)')
right_layer = geemap.ee_tile_layer(worldcover2021, {}, 'ESA Worldcover 2021 (10 m)')

Map.split_map(left_layer, right_layer)

Map.add_legend(title='ESA Globcover 2009 (300 m)', legend_dict=legend_dict, position='bottomleft')
Map.add_legend(title="ESA WorldCover 2021 (10 m)", builtin_legend="ESA_WorldCover", position='bottomright')
